In [14]:
import kaggle

In [15]:
! kaggle datasets init -p /app/_data/models/final/eff5WOComplex/

Data package template written to: /app/_data/models/final/eff5WOComplex/dataset-metadata.json


In [16]:
! kaggle datasets create -p /app/_data/models/final/eff5WOComplex/

Starting upload for file eff5_without_complex_3.h5
100%|█████████████████████████████████████████| 327M/327M [08:31<00:00, 670kB/s]
Upload successful: eff5_without_complex_3.h5 (327MB)
Starting upload for file eff5_without_complex_4.h5
100%|█████████████████████████████████████████| 327M/327M [08:31<00:00, 671kB/s]
Upload successful: eff5_without_complex_4.h5 (327MB)
Starting upload for file eff5_without_complex_1.h5
100%|█████████████████████████████████████████| 327M/327M [08:29<00:00, 673kB/s]
Upload successful: eff5_without_complex_1.h5 (327MB)
Starting upload for file eff5_without_complex_5.h5
100%|█████████████████████████████████████████| 327M/327M [09:00<00:00, 635kB/s]
Upload successful: eff5_without_complex_5.h5 (327MB)
Starting upload for file eff5_without_complex_2.h5
100%|█████████████████████████████████████████| 327M/327M [08:42<00:00, 657kB/s]
Upload successful: eff5_without_complex_2.h5 (327MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
You

In [3]:
!pip install albumentations

     |████████████████████████████████| 72 kB 412 kB/s eta 0:00:011
     |████████████████████████████████| 948 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 30.2 MB 16.1 MB/s eta 0:00:01     |█████████████████████████▎      | 23.8 MB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 51.0 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 32.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 165 kB 18.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import albumentations
import matplotlib.pyplot as plt

%matplotlib inline
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

In [5]:
PATH = "/app/_data/"
BATCH_SIZE = 26
SEED = 37
IMAGE_SIZE = 456
NUM_CLASSES = 6
TRAIN_IMG_PATH = "/app/_data/456_npy/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = [
    "complex",
    "frog_eye_leaf_spot",
    "healthy",
    "powdery_mildew",
    "rust",
    "scab",
]

In [6]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
wrong = [
    "ead085dfac287263.jpg",
    "95276ccd226ad933.jpg",
    "da8770e819d2696d.jpg",
    "cd3a1d64e6806eb5.jpg",
    "ccec54723ff91860.jpg",
]
df_labels = pd.read_csv("/app/_data/df_csv/labels_21_20.csv", index_col=[0])
df_labels = df_labels.query("image not in @wrong").reset_index(drop=True)
df_labels["image"] = df_labels["image"].str.replace(".jpg", ".npy")
df_labels = df_labels.sample(frac=1, random_state=SEED).reset_index(drop=True)
df_complex = df_labels.query('labels == "complex"').reset_index(drop=True)
df_without_complex = df_labels.query('labels != "complex"').reset_index(drop=True)

In [8]:
df_without_complex["image"]

0        9888ce2d3e363acd.npy
1        901f704f717817b9.npy
2        91673f856431749b.npy
3        f09047534b76db29.npy
4        9005d36127b0f77b.npy
                 ...         
18643          Train_1595.npy
18644    afa23b9c5aa958a8.npy
18645    a194c734b2996bad.npy
18646    86f636c5e84e23cc.npy
18647    8fbc33c7895a330c.npy
Name: image, Length: 18648, dtype: object

In [9]:
# for 380*380
transform = albumentations.Compose(
    [
        albumentations.CLAHE(p=0.1, clip_limit=(1, 1), tile_grid_size=(8, 8)),
        albumentations.OneOf(
            [
                albumentations.MotionBlur((3, 3)),
                albumentations.MedianBlur(blur_limit=3),
                albumentations.GaussianBlur(blur_limit=(3, 3), sigma_limit=0),
                albumentations.Blur(blur_limit=(3, 3)),
            ],
            p=0.2,
        ),
        albumentations.OneOf(
            [
                albumentations.GaussNoise(var_limit=[10, 20], mean=1),
                albumentations.ISONoise(
                    intensity=(0.1, 0.2), color_shift=(0.001, 0.005)
                ),
                albumentations.ImageCompression(
                    quality_lower=90, quality_upper=100, compression_type=1
                ),
                albumentations.MultiplicativeNoise(
                    multiplier=(0.98, 1.02), per_channel=True, elementwise=True
                ),
                albumentations.Downscale(
                    scale_min=0.9, scale_max=0.99, interpolation=4
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=(-5, 5),
                    sat_shift_limit=(-5, 5),
                    val_shift_limit=(-5, 5),
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=0.1,
                    contrast_limit=0.1,
                    brightness_by_max=True,
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.OpticalDistortion(
                    distort_limit=0.05,
                    shift_limit=0.05,
                    border_mode=2,
                ),
                albumentations.ElasticTransform(
                    alpha=2.0,
                    sigma=50.0,
                    alpha_affine=10.0,
                    interpolation=0,
                    border_mode=2,
                ),
                albumentations.GridDistortion(
                    num_steps=5, distort_limit=0.1, interpolation=0, border_mode=2
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HorizontalFlip(),
                albumentations.VerticalFlip(),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.Rotate(
                    limit=(-180, 180), interpolation=0, border_mode=2
                ),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=180,
                    interpolation=0,
                    border_mode=2,
                ),
            ],
            p=0.5,
        ),
    ]
)

In [10]:
class Generator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        images_src_dir,
        batch_size,
        target_image_size,
        shuffle=False,
        augment=True,
        crop=False,
        resize=False,
        normalize=False,
    ):
        self.shuffle = shuffle
        self.batch_size = BATCH_SIZE
        self.df = df
        self.images_dir = images_src_dir
        self.target_image_size = (IMAGE_SIZE, IMAGE_SIZE)
        self.augment = augment
        self.crop = crop
        self.resize = resize
        self.normalize = normalize
        # create label index map
        self.labels = self._read_labels()
        self.n_samples = self.df.shape[0]
        self.n_batches = self.n_samples // self.batch_size
        # shuffle data, also repeated after each epoch if needed
        if self.shuffle:
            np.random.shuffle(self.labels)

    def _read_labels(self):
        """
        Returns list images mapping to 1-hot label
        """

        # label indexes
        label_ixs = self.df[feature_columns].values
        image_ixs = self.df["image"].values
        labels = []

        for i in range(len(image_ixs)):
            labels.append([image_ixs[i], label_ixs[i]])
        return labels

    def __len__(self):
        """
        Length in batches
        """
        return self.n_batches

    def __getitem__(self, b_ix):
        """
        Produce batch, by batch index
        """

        assert b_ix < self.n_batches

        b_X = np.zeros(
            (self.batch_size, self.target_image_size[0], self.target_image_size[1], 3),
            dtype=np.uint8,
        )

        b_Y = np.zeros(
            (self.batch_size, self.df[feature_columns].shape[1]),
            dtype=np.uint8,
        )

        for i in range(self.batch_size):
            b_X[i], b_Y[i] = self.get_one(
                i + self.batch_size * b_ix,
            )

        return (b_X, b_Y)

    def get_one(self, one_ix):
        """
        Get single item by absolute index
        """
        id = self.labels[one_ix][0]
        src_file = self.images_dir + id

        # read file
        x = np.load(src_file)
        if self.crop:
            coord = self.df[self.df["image"] == id][
                ["x_min", "y_min", "x_max", "y_max"]
            ].values[0]
            orig_hight = x.shape[0]
            orig_width = x.shape[1]
            x_min = coord[0]
            y_min = coord[1]
            x_max = coord[2]
            y_max = coord[3]
            x = x[
                np.int(y_min * orig_hight) : np.int(y_max * orig_hight),
                np.int(x_min * orig_width) : np.int(x_max * orig_width),
            ]

        y = self.labels[one_ix][1]

        # augment
        if self.augment:
            x = self._augment_image(x)

        # normalize (sample-wise)
        if self.normalize:
            x = x.astype(np.float32)
            x = x - np.mean(x, axis=(0, 1))
            x = x / np.std(x, axis=(0, 1))
        return x.astype(np.uint8), y

    def _augment_image(self, x):
        """
        Randomply augment image
        """

        x = transform(image=x)["image"]
        return x

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.labels)

In [11]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [12]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB5(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b5_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [11]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_without_complex["image"], df_without_complex["labels"])
):
    train, valid = (
        df_without_complex.loc[train_index],
        df_without_complex.loc[valid_index],
    )
    model_name = "eff5_without_complex_" + str(i + 1) + ".h5"
    log_dir = "logs_eff5_wo_complex_" + str(i + 1) + "/"
    print('\nStart of #'+str(i+1)+' iteration\n')

    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_f1_score",
            patience=12,
            restore_best_weights=True,
            verbose=1,
            mode="max",
        ),
        keras.callbacks.ModelCheckpoint(
            "/app/_data/models/final/eff5WOComplex/" + model_name,
            monitor="val_f1_score",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_f1_score",
            factor=0.9,
            patience=8,
            verbose=1,
            mode="max",
            min_delta=1e-4,
            min_lr=0.00000001,
        ),
        keras.callbacks.TensorBoard(
            log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
        ),
        keras.callbacks.experimental.BackupAndRestore(
            "/app/_data/models/final/eff5WOComplex/backup/"
        ),
        keras.callbacks.TerminateOnNaN(),
    ]
    model = get_model()
    gen_train = Generator(
        df=train,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=True,
        crop=False,
        resize=False,
    )
    gen_valid = Generator(
        df=valid,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        augment=False,
        crop=False,
        resize=False,
    )

    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=100,
        steps_per_epoch=train.shape[0] // BATCH_SIZE,
        validation_steps=valid.shape[0] // BATCH_SIZE,
        verbose=1,
        workers=25,
        callbacks=callbacks,
    )
    tf.keras.backend.clear_session()


Start of #1 iteration

Epoch 1/100
  1/573 [..............................] - ETA: 0s - loss: 0.7612 - acc: 0.1154 - recall: 0.5000 - precision: 0.1346 - f1_score: 0.1031WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
573/573 [==============================] - ETA: 0s - loss: 0.1043 - acc: 0.8866 - recall: 0.8733 - precision: 0.9173 - f1_score: 0.8779
Epoch 00001: val_f1_score improved from -inf to 0.90456, saving model to /app/_data/models/final/eff5WOComplex/eff5_without_complex_1.h5
573/573 [==============================] - 384s 670ms/step - loss: 0.1043 - acc: 0.8866 - recall: 0.8733 - precision: 0.9173 - f1_score: 0.8779 - val_loss: 0.0840 - val_acc: 0.9250 - val_recall: 0.9059 - val_precision: 0.9342 - val_f1_score: 0.9046
Epoch 2/100
573/573 [========

In [11]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [28]:
def pred_to_labels(pred, thresh=0.5, labels=feature_columns):
    pred = [labels[i] for i in range(len(labels)) if pred[i] > thresh]
    pred = " ".join(pred)
    return pred


def predict_new(path, model):
    img = np.load(TRAIN_IMG_PATH + path)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    return pred_to_labels(pred[0])

In [20]:
model_list = [
    "eff5_without_complex_1.h5",
    "eff5_without_complex_2.h5",
    "eff5_without_complex_3.h5",
    "eff5_without_complex_4.h5",
    "eff5_without_complex_5.h5",
]

In [21]:
# _data/models/final/eff5WOComplex/eff5_without_complex_1.h5

In [22]:
df_sub = pd.DataFrame(columns=["image", "labels"])
for model_path in model_list:
    model = keras.models.load_model(
        "/app/_data/models/final/eff5WOComplex/" + model_path
    )
    for img_name in df_complex["image"].tolist():
        pred = predict_new(img_name, model)

        df_sub = df_sub.append({"image": img_name, "labels": pred}, ignore_index=True)

    print(df_sub.head())

                  image                           labels
0  fdc2bde10d847494.npy                     complex rust
1  a0f3ba197c2c86ce.npy  complex frog_eye_leaf_spot scab
2  d9096bd4a497b493.npy          frog_eye_leaf_spot rust
3  afe0c59036567467.npy       complex frog_eye_leaf_spot
4  c8c03ddfeb881f09.npy          frog_eye_leaf_spot scab
                  image                           labels
0  fdc2bde10d847494.npy                     complex rust
1  a0f3ba197c2c86ce.npy  complex frog_eye_leaf_spot scab
2  d9096bd4a497b493.npy          frog_eye_leaf_spot rust
3  afe0c59036567467.npy       complex frog_eye_leaf_spot
4  c8c03ddfeb881f09.npy          frog_eye_leaf_spot scab
                  image                           labels
0  fdc2bde10d847494.npy                     complex rust
1  a0f3ba197c2c86ce.npy  complex frog_eye_leaf_spot scab
2  d9096bd4a497b493.npy          frog_eye_leaf_spot rust
3  afe0c59036567467.npy       complex frog_eye_leaf_spot
4  c8c03ddfeb881f09.npy        

In [13]:
# df_sub.to_csv("/app/_data/df_csv/eff5_without_complex.csv")
df_sub = pd.read_csv("/app/_data/df_csv/eff5_without_complex.csv", index_col=[0])

In [15]:
df_sub = df_sub.join(df_sub["labels"].str.get_dummies(" "))

In [16]:
df_sub.shape[0] / 5

1577.0

In [17]:
len(df_sub[df_sub["complex"] == 1]) / 5

633.6

In [18]:
# df_sub.to_csv("/app/_data/df_csv/complex_predicted_kf_main.csv")
# df_sub = pd.read_csv("/app/_data/df_csv/complex_predicted_kf_main.csv", index_col=[0])

In [19]:
# df_sub['complex']=1

In [20]:
df_sub

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,fdc2bde10d847494.npy,complex rust,1,0,0,0,1,0
1,a0f3ba197c2c86ce.npy,complex frog_eye_leaf_spot scab,1,1,0,0,0,1
2,d9096bd4a497b493.npy,frog_eye_leaf_spot rust,0,1,0,0,1,0
3,afe0c59036567467.npy,complex frog_eye_leaf_spot,1,1,0,0,0,0
4,c8c03ddfeb881f09.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...
7880,f542c9ab3b30e48e.npy,complex frog_eye_leaf_spot scab,1,1,0,0,0,1
7881,c5dd359660ef0299.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1
7882,f5cb5a941a337605.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1
7883,d3a407fa4e23f09c.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1


In [21]:
df_pivot = df_sub.pivot_table(index="image", aggfunc=sum)

In [22]:
df_pivot

,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
image,,,,,,
800cbf0ff87721f8.npy,4,5,0,0,2,3
801f78399a44e7af.npy,3,5,0,0,0,3
802291cee9fec9f4.npy,2,5,0,0,0,5
803e3bd17a16e65c.npy,3,5,0,0,0,5
8052c7272b3b3f5c.npy,0,5,0,0,0,4
...,...,...,...,...,...,...
ffb084c28f4f30cc.npy,2,5,0,0,0,5
ffd4c5088f4605a7.npy,0,5,0,0,0,5
ffdc70c492370f02.npy,1,2,0,0,4,1


In [23]:
df_pivot[df_pivot <= 4] = 0

In [24]:
df_pivot[df_pivot > 4] = 1

In [25]:
df_pivot["complex"] = 1

In [29]:
for i in df_pivot.index.tolist():
    pred = pred_to_labels(df_pivot.loc[i].values)
    df_pivot.loc[i, "labels"] = pred

In [30]:
df_pivot.sample()

,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab,labels
image,,,,,,,
9eb188d3822f627e.npy,1,1,0,0,0,1,complex frog_eye_leaf_spot scab


In [31]:
df_pivot["labels"].value_counts()

complex frog_eye_leaf_spot         590
complex frog_eye_leaf_spot scab    472
complex scab                       169
complex rust                       162
complex                            143
complex powdery_mildew              28
complex frog_eye_leaf_spot rust      8
complex healthy                      5
Name: labels, dtype: int64

In [32]:
df_pivot["image"] = df_pivot.index
df_pivot = df_pivot.reset_index(drop=True)

In [33]:
df_all = pd.concat([df_without_complex, df_pivot], axis=0, ignore_index=True)

In [34]:
df_all = df_all.sample(frac=1)

In [35]:
df_all

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
2863,9c286e3f35303573.npy,scab,0,0,0,0,0,1
8138,e80f787d0170e937.npy,rust,0,0,0,0,1,0
866,e690cbcb851e95b8.npy,scab,0,0,0,0,0,1
10278,fe59c2f00f61742c.npy,healthy,0,0,1,0,0,0
9078,83a56fde95254a91.npy,scab,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
20184,fd9790d658283ce1.npy,complex frog_eye_leaf_spot scab,1,1,0,0,0,1
5788,f435ce08387b46e9.npy,powdery_mildew,0,0,0,1,0,0
10847,Train_214.npy,scab,0,0,0,0,0,1
1502,97abdad2485a4b25.npy,healthy,0,0,1,0,0,0


In [36]:
tf.keras.backend.clear_session()

# StratifiedShuffleSplit

In [40]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

In [43]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB5(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b5_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [46]:
        tf.keras.backend.clear_session()

In [37]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_all["image"], df_all["labels"])
):
    if i>=2:
        train, valid = (
            df_all.loc[train_index],
            df_all.loc[valid_index],
        )
        model_name = "eff5_pred_complex_" + str(i + 1) + ".h5"
        log_dir = "logs_eff5_pred_complex_" + str(i + 1) + "/"
        print('\nStart of #'+str(i+1)+' iteration\n')

        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor="val_f1_score",
                patience=12,
                restore_best_weights=True,
                verbose=1,
                mode="max",
            ),
            keras.callbacks.ModelCheckpoint(
                "/app/_data/models/final/eff5WOComplex/" + model_name,
                monitor="val_f1_score",
                verbose=1,
                save_best_only=True,
                save_weights_only=False,
                mode="max",
                save_freq="epoch",
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor="val_f1_score",
                factor=0.9,
                patience=8,
                verbose=1,
                mode="max",
                min_delta=1e-4,
                min_lr=0.00000001,
            ),
            keras.callbacks.TensorBoard(
                log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
            ),
            keras.callbacks.experimental.BackupAndRestore(
                "/app/_data/models/final/eff5WOComplex/backup/"
            ),
            keras.callbacks.TerminateOnNaN(),
        ]
        model = get_model()
        gen_train = Generator(
            df=train,
            images_src_dir=TRAIN_IMG_PATH,
            target_image_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            shuffle=True,
            augment=True,
            crop=False,
            resize=False,
        )
        gen_valid = Generator(
            df=valid,
            images_src_dir=TRAIN_IMG_PATH,
            target_image_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            shuffle=False,
            augment=False,
            crop=False,
            resize=False,
        )

        history = model.fit(
            gen_train,
            validation_data=gen_valid,
            epochs=100,
            steps_per_epoch=train.shape[0] // BATCH_SIZE,
            validation_steps=valid.shape[0] // BATCH_SIZE,
            verbose=1,
            workers=25,
            callbacks=callbacks,
        )
        tf.keras.backend.clear_session()


Start of #3 iteration

Epoch 27/100
  1/622 [..............................] - ETA: 0s - loss: 0.0177 - acc: 0.9615 - recall: 1.0000 - precision: 0.9677 - f1_score: 0.9225WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
622/622 [==============================] - ETA: 0s - loss: 0.0333 - acc: 0.8990 - recall: 0.9650 - precision: 0.9709 - f1_score: 0.8984
Epoch 00027: val_f1_score improved from -inf to 0.91164, saving model to /app/_data/models/final/eff5WOComplex/eff5_pred_complex_3.h5
622/622 [==============================] - 383s 615ms/step - loss: 0.0333 - acc: 0.8990 - recall: 0.9650 - precision: 0.9709 - f1_score: 0.8984 - val_loss: 0.0251 - val_acc: 0.9166 - val_recall: 0.9749 - val_precision: 0.9806 - val_f1_score: 0.9116
Epoch 28/100
622/622 [=========

In [ ]:
"cd3a1d64e6806eb5.jpg", "ccec54723ff91860.jpg", ""

In [38]:
import kaggle

In [39]:
! kaggle datasets init -p /app/_data/models/final/Eff5PredComplex/

Data package template written to: /app/_data/models/final/Eff5PredComplex/dataset-metadata.json


In [40]:
! kaggle datasets create -p /app/_data/models/final/Eff5PredComplex/

Starting upload for file eff5_pred_complex_3.h5
100%|█████████████████████████████████████████| 327M/327M [07:56<00:00, 721kB/s]
Upload successful: eff5_pred_complex_3.h5 (327MB)
Starting upload for file eff5_pred_complex_1.h5
100%|█████████████████████████████████████████| 327M/327M [07:59<00:00, 715kB/s]
Upload successful: eff5_pred_complex_1.h5 (327MB)
Starting upload for file eff5_pred_complex_5.h5
100%|█████████████████████████████████████████| 327M/327M [07:55<00:00, 721kB/s]
Upload successful: eff5_pred_complex_5.h5 (327MB)
Starting upload for file eff5_pred_complex_4.h5
100%|█████████████████████████████████████████| 327M/327M [07:50<00:00, 729kB/s]
Upload successful: eff5_pred_complex_4.h5 (327MB)
Starting upload for file eff5_pred_complex_2.h5
100%|█████████████████████████████████████████| 327M/327M [08:05<00:00, 707kB/s]
Upload successful: eff5_pred_complex_2.h5 (327MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Your private Dataset is being cre